In [ ]:
import pandas as pd
import numpy as np

from pprint import pprint

from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score, roc_curve, auc, precision_recall_curve, average_precision_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split, cross_val_score

from sklearn.ensemble import RandomForestClassifier

import import_ipynb
import algocryptos_preprocessing as alg_preproc
from utils_csa import save_obj, load_obj #, evaluate_model, evaluate_model_formated, show_model_accuracy

# from sklearn.metrics import SCORERS
# print(SCORERS.keys())

In [ ]:
dict_hours_labels  = {24:'1d'}
predict_only_one_crypto = True
y_to_be_considered = 'y_+1d_classif'
close_price_increase_targeted = +5
threshold = 0.85
id_cryptocompare_1 = "7605"

dict_df = load_obj('dict_df_2018_12_03_top10')

X_train_1, X_test_1, y_train_1, y_test_1, X_train_close_price_1, X_test_close_price_1 = alg_preproc.get_preprocessed_data(dict_df, dict_hours_labels, 
                                                                     close_price_increase_targeted, 
                                                                     False,
                                                                     do_scale=False, 
                                                                     do_pca=True,
                                                                     id_cryptocompare=id_cryptocompare_1)

In [22]:
n_iter = 1000
cv = 5

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 500, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 25, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
criterion=['entropy', 'gini']

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'criterion': criterion}

scorers = {
        'f1_score': make_scorer(f1_score)
    }

def start_tuning():   
    clf = RandomForestClassifier(random_state=0, 
                                 class_weight='balanced')
    rf_random = RandomizedSearchCV(estimator = clf, param_distributions = random_grid, n_iter = n_iter, verbose=20, cv = cv, random_state=0, n_jobs = -1,
                                  scoring=scorers, refit='f1_score', return_train_score=True)
    rf_random.fit(X_train_1, y_train_1[y_to_be_considered])

    # rf_random.cv_results_
    # rf_random.best_estimator_
    # rf_random.best_params_
    print('rf best_estimator_: ')
    print(rf_random.best_estimator_)
    print('\n rf best_params_: ')
    print(rf_random.best_params_)

In [ ]:
#rf_random.best_estimator_

# show_model_accuracy('RandomForestClassifier - Train', rf_random.best_estimator_, X_train_1, y_train_1[y_to_be_considered], None, do_precision_recall_curve=True, do_features_importance=True, threshold=0.5)
# show_model_accuracy('RandomForestClassifier - Test', rf_random.best_estimator_, X_test_1, y_test_1[y_to_be_considered], None, do_precision_recall_curve=True, do_precision_recall_vs_treshold=True, do_features_importance=False, threshold=0.5)


In [ ]:
# from sklearn.model_selection import cross_val_score
# print(cross_val_score(rf_random.best_estimator_, X_train_1, y_train_1[y_to_be_considered], cv=5, scoring='f1', n_jobs=-1))